In [ ]:
import datetime
import numpy as np
import pandas as pd
from tkinter import *
from tkinter import ttk
import plotly.express as px
from tkinter import messagebox
from cryptocmd import CmcScraper
import plotly.graph_objects as go
from IPython.display import clear_output
from IPython.display import display, HTML

master_df = None

#Logical Part
def generate():
    clear_output(wait=True)
    crypto_currency = crypto_currency_choice.get().split('-')[0].strip()
    crypto_symbol = crypto_currency_choice.get().split('-')[-1].strip()
    indicator_name = indicator_choice.get()
    get_price_dataframe(crypto_symbol)
    get_price_chart(crypto_currency)
    if indicator_name == 'RSI':
        calculate_rsi(crypto_currency)
    elif indicator_name == 'MACD':
        calculate_macd(crypto_currency)
    else:
        calculate_4ema(crypto_currency)
    
date_today = datetime.date.today()
date_one_year_back = date_today.replace(date_today.year - 1)

date_today_string = date_today.strftime('%d-%m-%Y')
date_one_year_back_string = date_one_year_back.strftime('%d-%m-%Y')

#Get 1 year historical data for selected crypto currency
def get_price_dataframe(crypto_symbol):
    global master_df
    
    #Call function to get historical data in dataframe
    scraper = CmcScraper(crypto_symbol, date_one_year_back_string, date_today_string)
    df = scraper.get_dataframe()
    
    #Set date as index
    df = df.set_index(pd.DatetimeIndex(df['Date'].values))
    
    #Assign to global variable
    master_df = df
    
#Show Crypto Currency Chart
def get_price_chart(crypto_name):
    global master_df
    labels = {
        'index' : 'Date',
        'Close' : 'Close Price ($)'
    }
    fig = px.line(master_df, master_df.index, master_df['Close'], title=f'1Y Price chart for {crypto_name}', labels=labels)
    fig.update_layout(yaxis_tickformat = '$')
    fig.show()
    
#RSI
def calculate_rsi(crypto_name):
    global master_df
    df = master_df
    
    #Get difference of prices from prev day
    delta = df['Close'].diff(1)

    #Remove NaN
    delta = delta.dropna()

    #Get ups and downs
    up = delta.copy()
    down = delta.copy()

    up[up<0] = 0
    down[down>0] = 0

    #Setting time period of 14 for RSI
    period = 14

    #Average gains and losees
    avg_gains = up.rolling(window=period).mean()
    avg_losses = abs(down.rolling(window=period).mean())

    #Calculate RS
    rs = avg_gains / avg_losses

    #Calculate RSI
    rsi = 100.0 - (100.0 / (1.0 + rs))
    
    #Show RSI plot
    new_df = pd.DataFrame()
    new_df['Close'] = df['Close']
    new_df['RSI'] = rsi
    
    labels = {
        'index' : 'Date',
    }
    fig = px.line(new_df, new_df.index, new_df['RSI'], title=f'RSI Plot for {crypto_name}', labels=labels)
    fig.update_layout(yaxis_ticksuffix='.00%')
    fig.show()
    
    #Call function to generate trade signals based on rsi
    generate_signals_for_rsi(new_df, crypto_name)
    
#Trade Signals based on RSI
def generate_signals_for_rsi(rsi_df, crypto_name):
    rsi_df = rsi_df[::-1]
    flag = -1
    buying_signals = []
    selling_signals = []
    profit = 0
    investment_data = []
    for i in range(len(rsi_df)):
        if rsi_df['RSI'][i] <= 25:
            if flag != 1:
                buying_signals.append(rsi_df['Close'][i])
                selling_signals.append(np.nan)
                flag = 1
                date = rsi_df.index[i].strftime('%d-%m-%Y')
                action = 'Buy'
                price = rsi_df['Close'][i]
                profit = profit - price
                investment_data.append([date, action, price, profit])
            else:
                buying_signals.append(np.nan)
                selling_signals.append(np.nan)
        elif rsi_df['RSI'][i] >= 65:
            if flag == 1:
                buying_signals.append(np.nan)
                selling_signals.append(rsi_df['Close'][i])
                flag = 0
                date = rsi_df.index[i].strftime('%d-%m-%Y')
                action = 'Sell'
                price = rsi_df['Close'][i]
                profit = profit + price
                investment_data.append([date, action, price, profit])
            else:
                buying_signals.append(np.nan)
                selling_signals.append(np.nan)
        else:
            buying_signals.append(np.nan)
            selling_signals.append(np.nan)
            
    #Visualize 
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=rsi_df.index, y=buying_signals,
                        mode='markers',
                        marker_symbol='triangle-up',
                        name='Buy',
                        marker_color='green'))
    fig.add_trace(go.Scatter(x=rsi_df.index, y=selling_signals,
                        mode='markers',
                        marker_symbol='triangle-down',
                        name='Sell',
                        marker_color='red'))
    fig.add_trace(go.Scatter(x=rsi_df.index, y=rsi_df['Close'],
                        mode='lines',
                        name='Price',
                        line_color='#126e82',
                        opacity=0.35))
    fig.update_layout(title=f'Trade Signals based on RSI for {crypto_name}', yaxis_tickformat = '$')
    fig.show()
    
    if len(investment_data):
        investment_df = pd.DataFrame(investment_data)
        investment_df.columns = ['Date', 'Action', 'Price ($)', 'Profit ($)']
        investment_df.style.hide_index()
        display(HTML(investment_df.to_html(index=False, justify='center')))
        display(HTML("<style>.dataframe{width:70%;margin:auto!important}.dataframe th,td{text-align: center!important}</style>"))
    
#MACD
def calculate_macd(crypto_name):
    global master_df
    df = master_df
    
    #Calculate short term exponential moving average / ema
    short_ema = df['Close'].ewm(span=12, adjust=False).mean()

    #Calculate short term exponential moving average / ema
    long_ema = df['Close'].ewm(span=26, adjust=False).mean()

    #MACD line
    macd = short_ema - long_ema

    #Signal line
    signal = macd.ewm(span=9, adjust=False).mean()
    
    #Show MACD plot
    new_df = pd.DataFrame()
    new_df['MACD'] = macd
    new_df['Signal'] = signal
    new_df['Close'] = df['Close']
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=new_df.index, y=new_df['MACD'],
                        mode='lines',
                        name='MACD'))
    fig.add_trace(go.Scatter(x=new_df.index, y=new_df['Signal'],
                        mode='lines',
                        name='Signal'))
    fig.update_layout(title=f'MACD Plot for {crypto_name}')
    fig.show()
    
    #Call function to generate trade signals based on macd
    generate_signals_for_macd(new_df, crypto_name)
    
#Trade Signals based on MACD
def generate_signals_for_macd(macd_df, crypto_name):
    macd_df = macd_df[::-1]
    flag = -1
    profit = 0
    investment_data = []
    buying_signals = []
    selling_signals = []
    for i in range(len(macd_df)):
        if macd_df['MACD'][i] > macd_df['Signal'][i]:
            selling_signals.append(np.nan)
            if flag != 1:
                buying_signals.append(macd_df['Close'][i])
                flag = 1
                date = macd_df.index[i].strftime('%d-%m-%Y')
                action = 'Buy'
                price = macd_df['Close'][i]
                profit = profit - price
                investment_data.append([date, action, price, profit])
            else:
                buying_signals.append(np.nan)
        elif macd_df['MACD'][i] < macd_df['Signal'][i]:
            buying_signals.append(np.nan)
            if flag == 1:
                selling_signals.append(macd_df['Close'][i])
                flag = 0
                date = macd_df.index[i].strftime('%d-%m-%Y')
                action = 'Sell'
                price = macd_df['Close'][i]
                profit = profit + price
                investment_data.append([date, action, price, profit])
            else:
                selling_signals.append(np.nan)
        else:
            buying_signals.append(np.nan)
            selling_signals.append(np.nan)

    #Visualize 
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=macd_df.index, y=buying_signals,
                        mode='markers',
                        marker_symbol='triangle-up',
                        name='Buy',
                        marker_color='green'))
    fig.add_trace(go.Scatter(x=macd_df.index, y=selling_signals,
                        mode='markers',
                        marker_symbol='triangle-down',
                        name='Sell',
                        marker_color='red'))
    fig.add_trace(go.Scatter(x=macd_df.index, y=macd_df['Close'],
                        mode='lines',
                        name='Price',
                        line_color='#126e82',
                        opacity=0.35))
    fig.update_layout(title=f'Trade Signals based on MACD for {crypto_name}', yaxis_tickformat = '$')
    fig.show()
    
    if len(investment_data):
        investment_df = pd.DataFrame(investment_data)
        investment_df.columns = ['Date', 'Action', 'Price ($)', 'Profit ($)']
        investment_df.style.hide_index()
        display(HTML(investment_df.to_html(index=False, justify='center')))
        display(HTML("<style>.dataframe{width:70%;margin:auto!important}.dataframe th,td{text-align: center!important}</style>"))
    
#4EMA
def calculate_4ema(crypto_name):
    global master_df
    df = master_df
    
    #Calculate four moving averages
    length_1 = df['Close'].ewm(span=8, adjust=False).mean()
    length_2 = df['Close'].ewm(span=13, adjust=False).mean()
    length_3 = df['Close'].ewm(span=21, adjust=False).mean()
    length_4 = df['Close'].ewm(span=55, adjust=False).mean()
    
    #Show 4EMA plot
    new_df = pd.DataFrame()
    new_df['Close'] = df['Close']
    new_df['length_1'] = length_1
    new_df['length_2'] = length_2
    new_df['length_3'] = length_3
    new_df['length_4'] = length_4
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=new_df.index, y=new_df['length_1'], mode='lines', name='8', marker_color='blue'))
    fig.add_trace(go.Scatter(x=new_df.index, y=new_df['length_2'], mode='lines', name='13', marker_color='purple'))
    fig.add_trace(go.Scatter(x=new_df.index, y=new_df['length_3'], mode='lines', name='21', marker_color='orange'))
    fig.add_trace(go.Scatter(x=new_df.index, y=new_df['length_4'], mode='lines', name='55', marker_color='yellow'))
    fig.update_layout(title=f'4EMA Plot for {crypto_name}', yaxis_tickformat = '$')
    fig.show()
    
    #Call function to generate trade signals based on 4ema
    generate_signals_for_4ema(new_df, crypto_name)
    
#Trade Signals based on 4EMA
def generate_signals_for_4ema(ema_df, crypto_name):
    ema_df = ema_df[::-1]
    flag = -1
    profit = 0
    investment_data = []
    buying_signals = []
    selling_signals = []
    for i in range(len(ema_df)):
        if ema_df['length_1'][i] < ema_df['length_2'][i] and ema_df['length_2'][i] < ema_df['length_3'][i] and ema_df['length_3'][i] < ema_df['length_4'][i]:
            if flag != 1:
                flag = 1
                buying_signals.append(ema_df['Close'][i])
                selling_signals.append(np.nan)
                date = ema_df.index[i].strftime('%d-%m-%Y')
                action = 'Buy'
                price = ema_df['Close'][i]
                profit = profit - price
                investment_data.append([date, action, price, profit])
            else:
                buying_signals.append(np.nan)
                selling_signals.append(np.nan)
        elif ema_df['length_1'][i] > ema_df['length_2'][i] and ema_df['length_2'][i] > ema_df['length_3'][i] and ema_df['length_3'][i] > ema_df['length_4'][i]:
            if flag == 1:
                flag = 0
                selling_signals.append(ema_df['Close'][i])
                buying_signals.append(np.nan)
                date = ema_df.index[i].strftime('%d-%m-%Y')
                action = 'Sell'
                price = ema_df['Close'][i]
                profit = profit + price
                investment_data.append([date, action, price, profit])
            else:
                buying_signals.append(np.nan)
                selling_signals.append(np.nan)
        else:
            buying_signals.append(np.nan)
            selling_signals.append(np.nan)
            
    #Visualize 
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ema_df.index, y=buying_signals,
                        mode='markers',
                        marker_symbol='triangle-up',
                        name='Buy',
                        marker_color='green'))
    fig.add_trace(go.Scatter(x=ema_df.index, y=selling_signals,
                        mode='markers',
                        marker_symbol='triangle-down',
                        name='Sell',
                        marker_color='red'))
    fig.add_trace(go.Scatter(x=ema_df.index, y=ema_df['Close'],
                        mode='lines',
                        name='Price',
                        line_color='#126e82',
                        opacity=0.35))
    fig.update_layout(title=f'Trade Signals based on 4EMA for {crypto_name}', yaxis_tickformat = '$')
    fig.show()
    
    if len(investment_data):
        investment_df = pd.DataFrame(investment_data)
        investment_df.columns = ['Date', 'Action', 'Price ($)', 'Profit ($)']
        investment_df.style.hide_index()
        display(HTML(investment_df.to_html(index=False, justify='center')))
        display(HTML("<style>.dataframe{width:70%;margin:auto!important}.dataframe th,td{text-align: center!important}</style>"))

    
#GUI Part
root = Tk()
root.title('Your Crypto Trader')
root.resizable(width=FALSE, height=FALSE)
root.geometry('800x350')

top_frame = Frame(root, bg='#126e82', pady=20)
top_frame.pack(side=TOP, fill=X)

heading_label = Label(top_frame, text='Your Crypto Trader', font=('Lato', 24), bg='#126e82', fg='#ffffff')
heading_label.pack()

bottom_frame = Frame(root, bg='#fff', pady=20)
bottom_frame.pack(side=TOP, fill=X)

choose_crypto_label = Label(bottom_frame, text='1. Select a cryptocurrency', font=('Lato', 14), bg='#fff')
choose_crypto_label.pack(side=TOP, padx=20)

crypto_currency_choice = StringVar()
crypto_currency_choosen = ttk.Combobox(bottom_frame, width=50, textvariable=crypto_currency_choice, state='readonly')
crypto_currency_choosen.pack(side=TOP, ipady=5)

crypto_currency_choosen['values'] = [
    'Bitcoin - BTC', 
    'Ethereum - ETH', 
    'Binance Coin - BNB',
    'XRP - XRP',
    'Cardano - ADA'
]
crypto_currency_choosen.current(0)

choose_indicator_label = Label(bottom_frame, text='2. Select an indicator', font=('Lato', 14), bg='#fff')
choose_indicator_label.pack(side=TOP, padx=20, pady=(20,0))

indicator_choice = StringVar()
indicator_choosen = ttk.Combobox(bottom_frame, width=50, textvariable=indicator_choice, state='readonly')
indicator_choosen.pack(side=TOP, ipady=5)

indicator_choosen['values'] = ['RSI', 'MACD', '4EMA']
indicator_choosen.current(0)

submit_btn = Button(bottom_frame, text='SUBMIT', font=('Lato', 18), bg='#126e82', fg='#fff', command=generate)
submit_btn.pack(side=TOP, ipady=5, pady=20)

root.mainloop()

Date,Action,Price ($),Profit ($)
17-04-2020,Buy,7096.184659,-7096.184659
13-04-2021,Sell,63503.457930,56407.273271
14-04-2021,Buy,63109.695935,-6702.422663
